In [6]:
import wandb
import os
import torch

# Initialize a W&B run
run = wandb.init(project="T20I-CRICKET-WINNER-PREDICTION")

# Use the `best_model` artifact
artifact = run.use_artifact('ravikumarchavva-org/T20I-CRICKET-WINNER-PREDICTION/best_model_val_loss_0.1193:v0', type='model')

# Download the model file locally
model_path = artifact.get_entry('best_model.pth').download()

# Load the model into memory using PyTorch
model = torch.load(model_path)

# Optional: Remove the downloaded file if it's no longer needed
os.remove(model_path)

# Print confirmation
print("Model loaded successfully!")

# Finish the W&B run
run.finish()

C:\Users\chavv\AppData\Local\Temp\ipykernel_7556\2644094845.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


Model loaded successfully!


In [7]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),".."))

from model_utils import collate_fn_with_padding, extract_data, augument_data, load_datasets
from torch.utils.data import DataLoader

# Load the Datasets
train_dataset, test_dataset, val_dataset = load_datasets()

# Step 2: Augment Data
# train_dataset, test_dataset, val_dataset = augument_data(train_dataset, test_dataset, val_dataset)

# Step 4: Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_with_padding)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_with_padding)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_with_padding)

# Step 5: Extract Data
train_team_data, train_player_data, train_ball_data, train_labels = extract_data(train_dataset)

Train Dataset: 824 samples
Validation Dataset: 103 samples
Test Dataset: 104 samples


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [15]:
for file in artifact.files():
    print(file.name)

best_model.pth


In [19]:
from model_utils import initialize_model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = artifact.metadata
print(config)
config['device'] = device
config['batch_size'] = 32
m = initialize_model(config,train_dataset,device)


# load the model from state dict
m.load_state_dict(model)

{'lr': 0.001, 'dropout': 0.5, 'num_epochs': 50, 'num_layers': 2, 'hidden_size': 128, 'weight_decay': 0.0001}


<All keys matched successfully>

In [20]:
m.eval()
all_labels = []
all_predictions = []
all_probs = []


# Define window sizes
stage_metrics = {}
with torch.no_grad():
    correct = 0
    total = 0
    for team, player, ball, labels in test_dataloader:
        team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)  # Move data to device
        team = team.float()
        player = player.float()
        ball = ball.float()
        labels = labels.float()
        
        outputs = m(team, player, ball)
        probs = outputs.squeeze().cpu().numpy()
        if probs.ndim == 0:
            probs = probs.reshape(1)  # Convert scalar to 1D array
        predicted = (outputs.data > 0.5).float()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        all_probs.extend(probs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Test Accuracy: {:.2f} %'.format(100 * correct / total))

Test Accuracy: 92.23 %
